### General rules:
 * For all figures that you generate, remember to add meaningful labels to the axes, and make a legend, if applicable.
 * Do not hard code constants, like number of samples, number of channels, etc in your program. These values should always be determined from the given data. This way, you can easily use the code to analyse other data sets.
 * Do not use high-level functions from toolboxes like scikit-learn.
 * Replace *Template* by your *FirstnameLastname* in the filename, or by *Lastname1Lastname2* if you work in pairs.

# BCI-IL - Exercise Sheet #10

#### Name:

In [6]:
% matplotlib inline

import numpy as np
import scipy as sp
import scipy.signal
from matplotlib import pyplot as plt

import bci_minitoolbox as bci
import bci_classifiers as cfy

## Preparation: Load data
*Note: As SSD and TDSEP are independent of labels, the markers are obsolete for the actual tasks, but you can use them to plot results for the different classes in the end if you want.*

In [12]:
fname = 'relaxVPis.npz'
cnt, fs, clab, mnt, mrk_pos, mrk_class, mrk_className = bci.load_data(fname)

cnt = cnt.T
print(cnt.shape)

(52, 34780)


## Exercise 1: SSD decomposition (6 points)
Implement a function SSD that performs a SSD decomposition of continuous EEG data for a given central and two flanking frequency bands. Design a butterworth band-pass filter of order N=*filterorder* with the frequency *center_band* (function `sp.signal.butter`) for the central fequency band and design similarly two sets of band-pass filters  of the same order for the flanking frequency bands *flank1_band* and *flank2_band*.
The function's output should be: $\bf W$ and $\bf A$ containing the filters and patterns in their columns, respectively, $\bf D$ is a diagonal matrix with the eigenvalues of the filters on the diagonal and $\bf Y$ is the data projected on the filters.

In [34]:
def train_SSD(cnt, center_band, flank1_band, flank2_band, filterorder, fs):
    ''' Usage: W, A, D, Y = trainSSD(cnt, center_band, flank1_band, flank2_band, filterorder, fs) '''
    
    
    # Get the BP filtered coefficients
    N = filterorder
        
    # Get filter coefficients    
    b_f, a_f = sp.signal.butter(N, np.array(center_band) / fs * 2,btype='bandpass')
    b_f1, a_f1 = sp.signal.butter(N, np.array(flank1_band) / fs * 2,btype='bandpass')    
    b_f2, a_f2 = sp.signal.butter(N, np.array(flank2_band) / fs * 2,btype='bandpass')
    
    # Define the Signal and noise Measurements     
    Ms  = sp.signal.lfilter(b_f, a_f, cnt)         
    X_f1 = sp.signal.lfilter(b_f1, a_f1, cnt)
    X_f2 = sp.signal.lfilter(b_f2, a_f2, cnt)
    Mn = ( X_f1 + X_f2 ) / 2

    C_s = np.cov(Ms)
    C_n = np.cov(Mn)

    D, V = sp.linalg.eig(C_s)

    C_s_r = V.T * C_s * V
    C_n_r = V.T * C_n * V 


    d, W = sp.linalg.eig(C_s_r, C_s_r + C_n_r )

    Y = V * W  

    A = C_s * W / (W.T* C_s * W)
    
    D = np.diag(d)
    
    return W, A, D, Y 
    
    
    
    
    

In [35]:
# N = 5
 
# # Define the Frequency bands    
# center_band = [11.,15.]
# flank1_band = [8.,10.]
# flank2_band = [16., 18.]
    
# # Get filter coefficients    
# b_f, a_f = sp.signal.butter(N, np.array(center_band) / fs * 2,btype='bandpass')
# b_f1, a_f1 = sp.signal.butter(N, np.array(flank1_band) / fs * 2,btype='bandpass')    
# b_f2, a_f2 = sp.signal.butter(N, np.array(flank2_band) / fs * 2,btype='bandpass')
    
# # Define the Signal and noise Measurements     
# Ms  = sp.signal.lfilter(b_f, a_f, cnt)         
# X_f1 = sp.signal.lfilter(b_f1, a_f1, cnt)
# X_f2 = sp.signal.lfilter(b_f2, a_f2, cnt)
# Mn = ( X_f1 + X_f2 ) / 2

# C_s = np.cov(Ms)
# C_n = np.cov(Mn)

# D, V = sp.linalg.eig(C_s)

# C_s_r = V.T * C_s * V
# C_n_r = V.T * C_n * V 


# D, W = sp.linalg.eig(C_s_r, C_s_r + C_n_r )

# Y = V * W  

# A = C_s * W / (W.T* C_s * W)


## Exercise 2 : TDSEP demixing (6 points)
Implement a function TDSEP that performs a TDSEP demixing on continuous EEG data, using the implementation for *two* time lags, as presented in the lecture. Output is the same as for the function SSD: $\bf W$ and $\bf A$ containing the filters and patterns in their columns, respectively, $\bf D$ is a diagonal matrix with the eigenvalues of the filters on the diagonal and $\bf Y$ is the data projected on the filters.

In [1]:
def train_TDSEP(cnt):
    ''' Usage: W, A, D, Y= train_TDSEP(cnt) '''
    # Pre-processing with a high pass filter
    N = 5 
    freq_filter = 0.04

    b, a = sp.signal.butter(N, freq_filter / fs * 2, btype='highpass')
    X = sp.signal.lfilter(b, a, cnt) 

    C_s = 0.5*(X[:,:-1]*X[:,1:].T + X[:,:-1].T*X[:,1:])

    D, W = sp.linalg.eig(C_s)

    Y = W.T * X 

    A = C_s * W / (W.T* C_s * W)

    D = np.diag(D)

    return W, A, D, Y

## Exercise 3: Validation (3 points)
Run the two algorithms on the data set.
For SSD, use 11-15 Hz for the central frequency band, 8-10 Hz and 16-18 Hz for the *two* flanking frequency bands with a filter order of *N=5*.
For TDSEP use [0 1] as the two time lags and preprocess the data before with a 0.04 Hz butterworth *high-pass* filter of order 5. 
For both methods, visualize the filter, pattern and projected data corresponding to the two smallest and the two largest Eigenvalues.

*Note: The actual digital filter order will be 10 for a bandpass with N=5. This is due to the fact that N is filter theoretic depicting the slope in the stop bands: a band-pass is a combined high and low-pass filter leading to the double digital order.*